In [41]:
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split

In [18]:
path = "sample_data/CharlesBookClub.csv"
df = pd.read_csv(path)

print(df.shape)
print(df.head())

(4000, 24)
   Seq#  ID#  Gender    M   R  F  FirstPurch  ChildBks  YouthBks  CookBks  \
0     1   25       1  297  14  2          22         0         1        1   
1     2   29       0  128   8  2          10         0         0        0   
2     3   46       1  138  22  7          56         2         1        2   
3     4   47       1  228   2  1           2         0         0        0   
4     5   51       1  257  10  1          10         0         0        0   

   ...  ItalCook  ItalAtlas  ItalArt  Florence  Related.Purchase  Mcode  \
0  ...         0          0        0         0                 0      5   
1  ...         0          0        0         0                 0      4   
2  ...         1          0        0         0                 2      4   
3  ...         0          0        0         0                 0      5   
4  ...         0          0        0         0                 0      5   

   Rcode  Fcode  Yes_Florence  No_Florence  
0      4      2             0 

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Seq#              4000 non-null   int64
 1   ID#               4000 non-null   int64
 2   Gender            4000 non-null   int64
 3   M                 4000 non-null   int64
 4   R                 4000 non-null   int64
 5   F                 4000 non-null   int64
 6   FirstPurch        4000 non-null   int64
 7   ChildBks          4000 non-null   int64
 8   YouthBks          4000 non-null   int64
 9   CookBks           4000 non-null   int64
 10  DoItYBks          4000 non-null   int64
 11  RefBks            4000 non-null   int64
 12  ArtBks            4000 non-null   int64
 13  GeogBks           4000 non-null   int64
 14  ItalCook          4000 non-null   int64
 15  ItalAtlas         4000 non-null   int64
 16  ItalArt           4000 non-null   int64
 17  Florence          4000 non-null  

In [21]:
#Split 60/40 na train/test
train_df, test_df = train_test_split(df, test_size=0.4, random_state=123, shuffle=True)

In [31]:
#sprawdzam czy rzeczywiście Florence jest 0/1
df["Florence"].dtype, df["Florence"].unique(), df["Florence"].value_counts(dropna=False)

(dtype('int64'),
 array([0, 1]),
 Florence
 0    3662
 1     338
 Name: count, dtype: int64)

In [32]:
# czy klient kupił Florence - odsetek jedynek = bazowy response rate
m = (train_df["Florence"] == 1).mean()
m


np.float64(0.08666666666666667)

In [34]:
#response rate dla każdej kombinacji RFM (grupy po Rcode,Fcode,Mcode):
rr = train_df.groupby(["Rcode","Fcode","Mcode"])["Florence"].mean().reset_index(name="rr")
rr


,Rcode,Fcode,Mcode,rr
0,1,1,1,0.000000
1,1,1,2,0.250000
2,1,1,3,0.000000
3,1,1,4,0.266667
4,1,1,5,0.107143
5,1,2,2,0.333333
6,1,2,3,0.000000
7,1,2,4,0.000000
8,1,2,5,0.043478
9,1,3,2,1.000000


Definicja segmentów 1–3 - jak w artykule:
•	Segment 1: rr > 2*m
•	Segment 2: m < rr <= 2*m
•	Segment 3: reszta


In [35]:
rr["segment"] = 3
rr.loc[rr["rr"] > 2*m, "segment"] = 1
rr.loc[(rr["rr"] > m) & (rr["rr"] <= 2*m), "segment"] = 2
rr


,Rcode,Fcode,Mcode,rr,segment
0,1,1,1,0.000000,3
1,1,1,2,0.250000,1
2,1,1,3,0.000000,3
3,1,1,4,0.266667,1
4,1,1,5,0.107143,2
5,1,2,2,0.333333,1
6,1,2,3,0.000000,3
7,1,2,4,0.000000,3
8,1,2,5,0.043478,3
9,1,3,2,1.000000,1


Licze, ilu klientów jest w każdej kombinacji RFM (n), doklejam do tego jej skuteczność (rr) i segment. Potem dla każdej kombinacji licze oczekiwaną liczbę odpowiedzi rr*n i sumuje to po segmentach. Segment z największą sumą ma największy potencjał “Florence==1” przy stałym koszcie kontaktu.

In [37]:
g = train_df.groupby(["Rcode","Fcode","Mcode"]).size().reset_index(name="n")
t = g.merge(rr, on=["Rcode","Fcode","Mcode"], how="left")

expected_by_segment = (t["rr"] * t["n"]).groupby(t["segment"]).sum()
best_segment = expected_by_segment.idxmax()

expected_by_segment, best_segment


(segment
 1     24.0
 2     77.0
 3    107.0
 dtype: float64,
 np.int64(3))

Biore tabelę rr,  każda kombinacja (Rcode,Fcode,Mcode) ma już policzony rr i przypisany segment. Filtruje tylko wiersze z segment == best_segment. Na końcu sklejam kody R/F/M w jeden string RFM (np. 1-1-4 → "114") i wypisuje liste tych kombinacji.

In [39]:
best_combos = rr.loc[rr["segment"] == best_segment, ["Rcode","Fcode","Mcode"]].copy()
best_combos["RFM"] = best_combos["Rcode"].astype(str) + best_combos["Fcode"].astype(str) + best_combos["Mcode"].astype(str)
best_combos[["RFM"]]


,RFM
0,111
2,113
6,123
7,124
8,125
15,213
16,214
17,215
26,312
27,313


każdemu klientowi w teście przypisuje „score” równy rr jego kombinacji RFM. Sortuje klientów od najwyższego score do najniższego i dziele tę listę na 10 równych części (decyle). W każdym decylu licze odsetek Florence==1, a lift to ten odsetek podzielony przez bazowy odsetek w całym teście. Jeśli lift w top decylu > 1, model sensownie wyłapuje lepszych klientów.

In [42]:
test_scored = test_df.merge(rr[["Rcode","Fcode","Mcode","rr"]], on=["Rcode","Fcode","Mcode"], how="left")
test_scored = test_scored.sort_values("rr", ascending=False).reset_index(drop=True)
test_scored["decyl"] = pd.qcut(test_scored.index + 1, 10, labels=False) + 1

base = test_scored["Florence"].mean()
lift_df = test_scored.groupby("decyl")["Florence"].mean().reset_index(name="resp_rate")
lift_df["percent_data"] = lift_df["decyl"] / 10
lift_df["lift"] = lift_df["resp_rate"] / base

fig = px.line(lift_df, x="percent_data", y="lift", markers=True, template="plotly_dark",
              title="Lift Chart of Segmentation Model",
              labels={"percent_data":"% of Customers Contacted", "lift":"Lift ratio"})
fig.add_hline(y=1)
fig.show()

lift_df[["percent_data","lift"]]


,percent_data,lift
0,0.1,1.538462
1,0.2,1.461538
2,0.3,1.076923
3,0.4,0.846154
4,0.5,1.000000
5,0.6,0.384615
6,0.7,0.769231
7,0.8,1.000000
8,0.9,1.000000
9,1.0,0.923077


In [40]:


# score = rr przypisane z kombinacji RFM
test_scored = test_df.merge(rr[["Rcode","Fcode","Mcode","rr"]], on=["Rcode","Fcode","Mcode"], how="left")

# sort malejąco po score
test_scored = test_scored.sort_values("rr", ascending=False).reset_index(drop=True)

# decyle (1 = top 10%)
test_scored["decyl"] = pd.qcut(test_scored.index + 1, 10, labels=False) + 1

# lift na test
base = test_scored["Florence"].mean()
lift_df = test_scored.groupby("decyl")["Florence"].mean().reset_index(name="resp_rate")
lift_df["lift"] = lift_df["resp_rate"] / base

# wykres
fig = px.line(lift_df, x="decyl", y="lift", markers=True, template="plotly_dark")
fig.show()

lift_df


,decyl,resp_rate,lift
0,1,0.12500,1.538462
1,2,0.11875,1.461538
2,3,0.08750,1.076923
3,4,0.06875,0.846154
4,5,0.08125,1.000000
5,6,0.03125,0.384615
6,7,0.06250,0.769231
7,8,0.08125,1.000000
8,9,0.08125,1.000000
9,10,0.07500,0.923077
